In [42]:
from google.cloud import storage
from pathlib import Path
from google.cloud import aiplatform
import tensorflow as tf
import os
import json
import numpy as np 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="SA.json"

In [19]:
project_id = 'premium-highway-370103'
bucket_name='flmodel'
bucket_location ='us'

storage_client = storage.Client(project=project_id)
bucket = storage_client.bucket(bucket_name)

In [24]:
!gcloud auth activate-service-account --key-file premium-highway-370103-78a48f5d3c42.json
!gsutil -m cp -r my_mnist_model gs://flmodel/

Activated service account credentials for: [flmodel@premium-highway-370103.iam.gserviceaccount.com]
If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying file://my_mnist_model/0001/fingerprint.pb [Content-Type=application/octet-stream]...
Copying file://my_mnist_model/0001/keras_metadata.pb [Content-Type=application/octet-stream]...
Copying file://my_mnist_model/0001/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://my_mnist_model/0001/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://my_mnist_model/0001/variables/variables.index [Content-Type=application/octet-stream]...
- [5/5 files][  2.1 MiB/  2.1 MiB] 100% Done 

In [29]:
server_image = "gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-8:latest"
vertexAI_location='us-central1'

aiplatform.init(project=project_id, location=vertexAI_location)
mnist_model = aiplatform.Model.upload(
    display_name="mnist",
    artifact_uri=f"gs://{bucket_name}/my_mnist_model/0001",
    serving_container_image_uri=server_image,
)

Creating Model
Create Model backing LRO: projects/192984306711/locations/us-central1/models/1438057855037997056/operations/1776409812339261440
Model created. Resource name: projects/192984306711/locations/us-central1/models/1438057855037997056@1
To use this Model in another session:
model = aiplatform.Model('projects/192984306711/locations/us-central1/models/1438057855037997056@1')


In [35]:
endpoint = aiplatform.Endpoint.create(display_name="mnist-endpoint")

endpoint.deploy(
    mnist_model,
    min_replica_count=1,
    max_replica_count=5,
    machine_type="n1-standard-4",
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=1
)

Creating Endpoint
Create Endpoint backing LRO: projects/192984306711/locations/us-central1/endpoints/8209116540696526848/operations/8579519261947461632
Endpoint created. Resource name: projects/192984306711/locations/us-central1/endpoints/8209116540696526848
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/192984306711/locations/us-central1/endpoints/8209116540696526848')
Deploying Model projects/192984306711/locations/us-central1/models/1438057855037997056 to Endpoint : projects/192984306711/locations/us-central1/endpoints/8209116540696526848
Deploy Endpoint model backing LRO: projects/192984306711/locations/us-central1/endpoints/8209116540696526848/operations/2515422363693088768


KeyboardInterrupt: 

In [38]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_new = X_test[:3]

In [39]:
def upload_directory(bucket, dirpath):
    dirpath = Path(dirpath)
    for filepath in dirpath.glob("**/*"):
        if filepath.is_file():
            blob = bucket.blob(filepath.relative_to(dirpath.parent).as_posix())
            blob.upload_from_filename(filepath)

In [40]:
response = endpoint.predict(instances=X_new.tolist())
np.round(response.predictions, 2)

AttributeError: 'NoneType' object has no attribute 'from_call'

In [43]:
batch_path = Path("my_mnist_batch")
batch_path.mkdir(exist_ok=True)
with open(batch_path / "my_mnist_batch.jsonl", "w") as jsonl_file:
    for image in X_test[:100].tolist():
        jsonl_file.write(json.dumps(image))
        jsonl_file.write("\n")

upload_directory(bucket, batch_path)

In [44]:
batch_prediction_job = mnist_model.batch_predict(
    job_display_name="my_batch_prediction_job",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=5,
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=1,
    gcs_source=[f"gs://{bucket_name}/{batch_path.name}/my_mnist_batch.jsonl"],
    gcs_destination_prefix=f"gs://{bucket_name}/my_mnist_predictions/",
    sync=True  # set to False if you don't want to wait for completion
)

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/192984306711/locations/us-central1/batchPredictionJobs/2294109158561021952
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/192984306711/locations/us-central1/batchPredictionJobs/2294109158561021952')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/2294109158561021952?project=192984306711
BatchPredictionJob projects/192984306711/locations/us-central1/batchPredictionJobs/2294109158561021952 current state:
JobState.JOB_STATE_PENDING
BatchPredictionJob projects/192984306711/locations/us-central1/batchPredictionJobs/2294109158561021952 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/192984306711/locations/us-central1/batchPredictionJobs/2294109158561021952 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/192984306711/locations/us-central1/batchPredictionJobs/

In [45]:
y_probas = []
for blob in batch_prediction_job.iter_outputs():
    if "prediction.results" in blob.name:
        for line in blob.download_as_text().splitlines():
            y_proba = json.loads(line)["prediction"]
            y_probas.append(y_proba)

In [49]:
y_pred = np.argmax(y_probas, axis=1)
accuracy = np.sum(y_pred == y_test[:100]) / 100
print(y_pred)
print(accuracy)

[2 0 2 5 1 6 7 1 8 0 1 4 2 9 2 1 1 6 1 4 6 5 7 4 5 5 3 4 1 2 2 8 0 2 5 7 9
 1 4 6 0 9 4 8 8 3 3 8 0 7 5 7 9 0 1 0 9 4 9 2 1 2 4 4 4 5 8 2 2 8 4 8 0 7
 7 8 5 1 1 0 4 7 8 9 0 2 6 4 3 1 2 8 4 1 8 5 9 5 0 3]
0.07


In [48]:
for prefix in ["my_mnist_model/", "my_mnist_batch/", "my_mnist_predictions/"]:
    blobs = bucket.list_blobs(prefix=prefix)
    for blob in blobs:
        blob.delete()

bucket.delete()  # if the bucket is empty
batch_prediction_job.delete()

NotFound: 404 GET https://storage.googleapis.com/storage/v1/b/flmodel/o?projection=noAcl&prefix=my_mnist_model%2F&prettyPrint=false: The specified bucket does not exist.